In [1]:
# pip install autograd

In [2]:
import ipywidgets as widgets
print(f'IPywidgets version: {widgets.__version__}')

import numpy as np
print(f'NumPy version: {np.__version__}')

import matplotlib.pyplot as plt
import matplotlib
print(f'Matplotlib version: {matplotlib.__version__}')

import pandas as pd
print(f'Pandas version: {pd.__version__}')

from autograd import elementwise_grad as egrad
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

IPywidgets version: 7.6.5
NumPy version: 1.21.5
Matplotlib version: 3.5.2
Pandas version: 1.4.4


# Ex 1
Reprezentati functia $\sin(k\cdot x) \cos(x)$ si derivata ei intr-un punct ales de catre utilizator; $k$ este de asemenea un punct ales de utilizator. Pentru calculul derivatei puteti folosi formula analitica sau biblioteca [autograd](https://github.com/HIPS/autograd).

In [3]:
import autograd.numpy as numpy

In [4]:
def function(x: float, k: float) -> float:
    """
    Computes the function sin(𝑘⋅𝑥)cos(𝑥) and returns its result.
    
    :param x: the given x parameter
    :param k: the given k parameter
    
    :return: the function result
    """
    return numpy.sin(k * x) * numpy.cos(x)

grad_function = egrad(function)  

In [5]:
def graph_f_and_derived(x, k, left=-5., right=5.):
    """
    Plots the function and its derived function in x and k.
    
    :param x: the chosen x
    :param k: the chosen k
    :param left: left margin 
    :param right: right margin
    
    :return: None
    """
    x_range: np.ndarray = np.linspace(left, right, 50)
    
    fx:np.ndarray = function(x_range, k)
    fdx: np.ndarray = grad_function(x_range, k)
    
    plt.figure(figsize=(5, 5))

    plt.plot(x_range, fx, color='red')
    plt.plot(x_range, fdx, color='blue')
    
    plt.scatter(x, function(x, k), color='red')
    plt.scatter(x, grad_function(x, k), color='blue')    
    

In [6]:
interact(graph_f_and_derived, x = (-5., 5.), k = (-5., 5.));

interactive(children=(FloatSlider(value=0.0, description='x', max=5.0, min=-5.0), FloatSlider(value=0.0, descr…

# Ex 2
Incarcati fisierul de date [Abalone](http://archive.ics.uci.edu/ml/datasets/Abalone) si in functie de alegerile exprimate de un utilizator in controale de tip dropdown, afisati intr-un grafic 2D coloanele numerice alese. Numele coloanelor se gasesc in fisierul `abalone.names`.

In [7]:
header_list: list = ['Sex', 'Length', 'Diam', 'Height',  'Whole', 'Shucked', 'Viscera', 'Shell', 'Rings']
data_frame: pd.DataFrame = pd.read_csv('abalone.data', header=None, names = header_list)
data_frame.head()

,Sex,Length,Diam,Height,Whole,Shucked,Viscera,Shell,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [8]:
groups = data_frame.groupby('Sex')
groups.mean()

,Length,Diam,Height,Whole,Shucked,Viscera,Shell,Rings
Sex,,,,,,,,
F,0.579093,0.454732,0.158011,1.046532,0.446188,0.230689,0.302010,11.129304
I,0.427746,0.326494,0.107996,0.431363,0.191035,0.092010,0.128182,7.890462
M,0.561391,0.439287,0.151381,0.991459,0.432946,0.215545,0.281969,10.705497


In [9]:
input1 = widgets.Dropdown(
    options=header_list[1:],
    description='Option 1'
)
input2 = widgets.Dropdown(
    options=header_list[1:],
    description='Option 2'
)

In [10]:
def plot_function(input1: str, input2: str) -> None:
    """
    Plots the data from the abalone dataset in a 2d plot using the two given inputs.
    
    :input1: first input
    :input2: second input
    
    """
    plt.figure(figsize=(10, 5))
    for name, group in groups:
        plt.plot(group[input1], group[input2], 'o', label = name)
        plt.xlabel(input1)
        plt.ylabel(input2)
    plt.legend()
    plt.show()

widgets.interact(plot_function, input1 = input1, input2 = input2);

interactive(children=(Dropdown(description='Option 1', options=('Length', 'Diam', 'Height', 'Whole', 'Shucked'…

# Ex 3
1. Folosind un API pentru date legate de vreme (vedeti o lista de API-uri ce se pot utiliza gratuit [aici](https://rapidapi.com/blog/access-global-weather-data-with-these-weather-apis/#:~:text=What%20is%20a%20Weather%20API,weather%20forecast%20and%20historical%20information.)), creati o interfata in Jupyter notebook sau Jupyterlab, folosind ipywidgets, in care sa expuneti valori preluate din API. Definiti controale adecvate, in functie de parametrii de apel permisi de API. 

In [11]:
import requests
import json
from ipywidgets import widgets

In [12]:
API_Key: str = "6276fddd42ced496cfea6c1f59e23087"

In [13]:
metric_dict = {'standard': 'Kelvin', 'metric': 'Celsius', 'imperial': 'Fahrenheit'}

In [14]:
cities = widgets.Dropdown(
    options = ['Paris', 'Brasov', 'Tokyo'],
    description = 'City'
)

units = widgets.Dropdown(
    options = list(metric_dict),
    description = 'Unit'
)

languages = widgets.Dropdown(
    options = ['en', 'ro', 'fr'],
    description = 'Language'
)

In [15]:
def print_weather(city_name: str, unit: str, lang: str) -> None:
    """
    Dislays the weather based on the given parameters.
    
    :param city_name: the selected city 
    :param unit: the unit for displaying the data 
    :param lang: the language parameter
    
    :return: None
    """
    if city_name is None:
        return
    
    url: str = f"http://api.openweathermap.org/data/2.5/weather?q={city_name}&units={unit}&lang={lang}&appid={API_Key}"
    
    response = requests.get(url)
    res = response.json()
    
    if res["cod"] != "404":
        data = res["main"]
        
        live_temperature = data["temp"]
        feels_like = data["feels_like"]
        temp_max = data["temp_max"]
        temp_min = data["temp_min"]
        humidity = data["humidity"]
        
        live_pressure = str(data["pressure"])
        
        weather = res["weather"]
        weather_description = str(weather[0]["description"])
        
        temp = widgets.Label(f"Temperature ({metric_dict[unit]}): {live_temperature}")
        temp_feel = widgets.Label(f"Feels like  ({metric_dict[unit]}): {feels_like}")
        temp_min = widgets.Label(f"Min temperature  ({metric_dict[unit]}): {temp_min}")
        temp_max = widgets.Label(f"Max temperature  ({metric_dict[unit]}): {temp_max}")
        humidity = widgets.Label(f"Humidity: {humidity}%")
        pressure = widgets.Label(f"Pressure: {live_pressure}hPa")
        description = widgets.Label(f"Description: {weather_description}")
        
        display(widgets.VBox([temp, temp_feel, temp_min, temp_max, humidity, pressure, description]))
    else:
        print("An error occured, please try later!")

In [16]:
widgets.interact(print_weather, city_name = cities, unit=units, lang=languages);

interactive(children=(Dropdown(description='City', options=('Paris', 'Brasov', 'Tokyo'), value='Paris'), Dropd…